# Atividade 3

Faça tudo via dbt. 

Envia como resposta:
- A query SQL
- Comando utilizado via terminal no `dbt`
- Um print do resultado no próprio notebook, para isso é necessário criar uma pasta no repositório onde serão armazenadas as imagens, e para inserir no notebook é necessário criar uma célula `markdown` e inserir o seguinte comando: 

```markdown
![Nome da Foto](path/para/imagem)
```

### 1. Crie uma tabela `orders_by_year` a partir de `orders` que agrupa a quantidade de pedidos por ano, considerando a coluna `order_approved_at`


```sql

--[query --> ./models/orders_by_year.sql ]

        {{ config(materialized='table') }}

        SELECT
            EXTRACT(YEAR FROM order_approved_at) AS order_approved_year,
            COUNT(order_id) AS num_orders
        FROM
            `dbt_shop.orders`
        GROUP BY
            1
```

```
--[comando  --> cd shop]

dbt run --models shop.orders_by_year
```

![Alt text](../image_query/1ta-questao.png)


### 2. Crie uma `view` que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.


```sql

--[query --> ./models/total_orders_by_state.sql]

        {{ config(materialized='view') }}

        SELECT
          `dbt_shop.custmers`.customer_state,
          COUNT(`dbt_shop.orders`.order_id) AS num_orders
        FROM
            `dbt_shop.custmers`
        JOIN
            `dbt_shop.orders` 
        ON
          `dbt_shop.custmers`.customer_id = `dbt_shop.orders`.customer_id
        GROUP BY
          1
```

```
--[comando --> cd shop]

      dbt run --models shop.total_orders_by_state
```

![Alt text](../image_query/2a-questao.png)

### 3. De acordo com o resultado do exercício 2 retorne apenas os estados que tem mais de 5000 pedidos, esse número deve ser passado via variável `number_of_orders` do `dbt`.


```sql

-- [query --> ./models/total_orders_by_state1.sql]

        {{config(materialized='view')}}

        SELECT
        `dbt_shop.custmers`.customer_state,
        COUNT(`dbt_shop.orders`.order_id) AS num_orders
        FROM
            `dbt_shop.custmers`
        JOIN
            `dbt_shop.orders` 
        ON
        `dbt_shop.custmers`.customer_id = `dbt_shop.orders`.customer_id
        GROUP BY
        1
        HAVING
        COUNT(`dbt_shop.orders`.order_id) > {{ var('number_of_orders') }}

```

```
--[comando --> cd shop]
     
       dbt run --models shop.total_orders_by_state1dbt run --models shop.total_orders_by_state1 --vars "{'number_of_orders': '5000'}"
```

![Alt text](../image_query/3a-questao.png)

### 4. Crie uma `view` chamada `customers_by_region` que utiliza os dados da tabela `customers`. A visão deve realizar uma agregação por região do Brasil (Norte, Nordeste, Sudeste, Centro Oeste e Sul) e contar o número de clientes em cada uma delas.


```sql

--[query --> ./models/customers_by_region.sql]
            {{config(materialized='view')}}

            SELECT
            CASE
                WHEN customer_state IN ('AC', 'AM', 'AP', 'PA', 'RO', 'RR', 'TO') THEN 'Norte'
                WHEN customer_state IN ('AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE') THEN 'Nordeste'
                WHEN customer_state IN ('ES', 'MG', 'RJ', 'SP') THEN 'Sudeste'
                WHEN customer_state IN ('DF', 'GO', 'MS', 'MT') THEN 'Centro-Oeste'
                WHEN customer_state IN ('PR', 'RS', 'SC') THEN 'Sul'
            END AS region,
            COUNT(DISTINCT customer_id) AS num_customers
            FROM
                `dbt_shop.custmers`
            GROUP BY
            1
```

```
--[comando --> cd shop]
        
            dbt run --models shop.customers_by_region
```

![Alt text](../image_query/4a-questao.png)

### 5. Crie uma tabela `items_costs` que contenha o preço total de um item (soma do valor do produto com o frete).


```sql

--[query  -> ./models/items_costs.sql]
        {{ config(materialized='table') }}

        SELECT
            `dbt_shop.items`.product_id as product,
            SUM(`dbt_shop.items`.price + `dbt_shop.items`.freight_value) as item_total_cost
        FROM
            `dbt_shop.items` 
        GROUP BY
            1
```

```
--[comando --> cd shop]

        dbt run --models shop.items_costs      
```

![Alt text](../image_query/5ta-questao.png)

### 6. Crie uma `view` com o nome `orders_shipped_costs`, você deve calcular o preço total do pedido somado com o frete, apenas para pedidos enviado (shipped).


```sql

--[query --> ./models/orders_shipped_costs]

         {{config(materialized='view')}}
      
         SELECT
               `dbt_shop.orders`.order_id,
               SUM(`dbt_shop.items`.price + `dbt_shop.items`.freight_value) AS order_total_cost
         FROM
               `dbt_shop.items`
            JOIN 
               `dbt_shop.orders`
            ON 
               `dbt_shop.items`.order_id = `dbt_shop.orders`.order_id
         WHERE
            `dbt_shop.orders`.order_status = 'shipped'
         GROUP BY
            `dbt_shop.orders`.order_id
```

```
--[comando --> cd shop]
 
      dbt run --models shop.orders_shipped_costs

```

![Alt text](../image_query/6a-questao.png)